In [14]:
import os
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import random
import glob
import nibabel as nib
import numpy as np
from scipy.ndimage import rotate
import csv
import SimpleITK as sitk
#from lungtumormask import mask as tumormask
#from lungmask import mask as lungmask_fun

from monai.utils import first, set_determinism
from monai.transforms import (
    RandFlipd,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    SqueezeDimd,
    LoadImaged,
    Orientationd,
    FillHoles,
    RemoveSmallObjects,
    KeepLargestConnectedComponent,
    RandCropByPosNegLabeld,
    SaveImaged,
    CenterSpatialCropd,
    SpatialCropd,
    ScaleIntensityRanged,
    Spacingd,
    AsDiscrete,
    SpatialCrop,
    RandSpatialCropd,
    SpatialPadd,
    EnsureTyped,
    EnsureType,
    Invertd,
    DivisiblePadd,
    MapTransform,
    RandWeightedCropd,
    ToTensord,
    Transpose,
    ToTensor,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet,VNet,SwinUNETR,UNETR,DynUNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric,SurfaceDiceMetric,SurfaceDistanceMetric,HausdorffDistanceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch,pad_list_data_collate
from monai.config import print_config
from monai.apps import download_and_extract

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

if False:
    %matplotlib inline

device: cuda


In [3]:
#peregrine
#root_path = '/data/p308104/NBIA_Data/NIFTI_NBIA/imagesTr/'
#root_path = '/data/p308104/Nifti_Imgs_V0/' #UMCG data on peregrine
#local
#root_path = '/home/umcg/Desktop/NBIA/NBIA_Nifti_v0/imagesTr/'
#root_path = '/home/umcg/Desktop/Dicom_UMCG/MultipleBP/' 
#root_path = '/home/umcg/OneDrive/MultipleBreathingP/'
root_path = '/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/'

all_patientdir = []
all_patientdir = os.listdir(root_path)
all_patientdir.sort()
print(len(all_patientdir))


214


In [4]:
CT_fpaths=[]
lbl_fpaths=[]
lung_fpaths=[]
patient_ok='X'
for patient_path in all_patientdir:
    ct_miss = True
    lung_miss = True
    gtv_miss = True
    for root, dirs, files in os.walk(root_path+patient_path, topdown=True):
        if len(files)>18:
            for f in files:
                if "_agtv" in f.lower():
                    lbl_fpaths.append(os.path.join(root_path,patient_path,f))
                    patient_ok = patient_path
        
    for root, dirs, files in os.walk(root_path+patient_path, topdown=True):
        for f in files:
            if patient_ok in f.lower():
                if "%_ct.nii.gz" in f.lower():
                    CT_fpaths.append(os.path.join(root_path,patient_path,f))
                if "%_lungmask.nii.gz" in f.lower():
                    lung_fpaths.append(os.path.join(root_path,patient_path,f))
        
print('Cts: ',len(CT_fpaths),len(lung_fpaths),len(lbl_fpaths))
CT_fpaths.sort()
lung_fpaths.sort()
lbl_fpaths.sort()

num = 1000
print(lung_fpaths[5])
print(CT_fpaths[5])
print(lbl_fpaths[5])



Cts:  10 10 10
/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0045632/0045632_50%_LungMask.nii.gz
/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0045632/0045632_50%_ct.nii.gz
/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0045632/0045632_50%_aGTV.nii.gz


In [5]:
for i in range(len(lung_fpaths)):
    lung_i = nib.load(lung_fpaths[i])
    ct_i = nib.load(CT_fpaths[i])
    gtv_i = nib.load(lbl_fpaths[i])
    
    print(lung_i.shape,ct_i.shape,gtv_i.shape)

(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)
(176, 512, 512) (512, 512, 176) (1, 288, 192, 192)


In [6]:
# Save csv file of patients ok and px not ok 
if False:
    print('List Ok: ',len(list_PxOk),'List NOT Ok',len(list_PxNOTOk))
    file = open('ListPatientsStatus.csv', 'w')
    writer = csv.writer(file)
    writer.writerow(['Patients Ok'])
    writer.writerow(list_PxOk)
    writer.writerow(['Patients NOT Ok'])
    writer.writerow(list_PxNOTOk)
    file.close()

In [7]:
#Predicts both tumor and lung mask
Predict = False
w = '50%'
if Predict:
    gtv = '/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0082998/0082998_rtstruct_ITV.nii.gz'
    gtv = nib.load(gtv)
    empty_header = nib.Nifti1Header()

    for ct in CT_fpaths:
        predicted_path = root_path+ct+'_'+w+'_predictedTS.nii.gz'
        lung_path = ct[:-10]+'_LungMask.nii.gz'
        #Get Tumor mask
        #tumormask.mask(ct, predicted_path)
        #Get Lung mask and save it
        input_image = sitk.ReadImage(ct, imageIO='NiftiImageIO')
        lungmask = lungmask_fun.apply(input_image)  # default model is U-net(R231)
        lungmask_ni = nib.Nifti1Image(lungmask, gtv.affine,empty_header)
        nib.save(lungmask_ni,lung_path)

In [8]:
#Check max min tumor values
if False :
    minmin = 10
    maxmax = 0
    for i in range(len(data_dicts)):
        img = nib.load(data_dicts[i]["lung"])
        data_img = img.get_fdata()
        data_img = np.transpose(data_img, (1,2,0))# fix misshape 
        for j in range(data_img.shape[2]):
            if np.sum(data_img[:,:,j])>0:
                plt.imshow(data_img[:,:,j])
                plt.show()
        break
        if np.min(data_img) < minmin:
          minmin = np.min(data_img)
        if np.max(data_img) > maxmax:
          maxmax = np.max(data_img)

    print("minmin: ",minmin,"maxmax: ", maxmax)


In [9]:
#Create data dictionary - ### ALL going for segmentation
data_dicts = [
    {"image": image_name,"lung":lung_name,"agtv":agtv_name}
    for image_name,lung_name,agtv_name in zip(CT_fpaths,lung_fpaths,lbl_fpaths)
]
val_files = data_dicts[:]
print('val len:',len(val_files))

minmin_CT = -1024 #NBIA
maxmax_CT = 3071 #NBIA
val_files[0]

val len: 10


{'image': '/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0045632/0045632_ 0%_ct.nii.gz',
 'lung': '/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0045632/0045632_ 0%_LungMask.nii.gz',
 'agtv': '/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0045632/0045632_ 0%_aGTV.nii.gz'}

In [10]:
#class to transpose lung mask
class Create_sequences(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
        print(f"keys to transpose: {self.keys}")

         
    def __call__(self, dictionary):
        dictionary = dict(dictionary)
        for key in self.keys:
            data = dictionary[key]
            if key == 'lung':
                data = np.transpose(data, (0,2,3,1))
                data = rotate(data,270,axes=(1,2),reshape=False)
                data = np.flip(data,1)
                data[data==2] = int(1)
                data[data!=1] = int(0)
            dictionary[key] = data
            
        return dictionary        

In [19]:
#Create Compose functions for preprocessing of train and validation
set_determinism(seed=0)
image_keys = ["image","lung"]
p = .5 #Data aug transform probability
size = 96
image_size = (size,size,size)

val_transforms = Compose(
    [
        LoadImaged(keys=image_keys),
        #SqueezeDimd(keys="agtv"),
        EnsureChannelFirstd(keys=image_keys),
        Orientationd(keys=["image"], axcodes="RAS"),
        #Spacingd(keys=["image","agtv"], pixdim=(1,1,1),mode=("bilinear","nearest")),
        ScaleIntensityRanged(keys=["image"], a_min=minmin_CT, a_max=maxmax_CT,b_min=0.0, b_max=1.0, clip=True,),
        Create_sequences(keys=image_keys),
        CropForegroundd(keys=image_keys, source_key="lung",k_divisible = size),
        ToTensord(keys=image_keys),
    ]
)


keys to transpose: ('image', 'lung')


In [27]:
#Create Compose functions for preprocessing of train and validation
set_determinism(seed=0)
image_keys = ["agtv"]
p = .5 #Data aug transform probability
size = 96
image_size = (size,size,size)

val_transforms = Compose(
    [
        LoadImaged(keys=image_keys),
        #SqueezeDimd(keys="agtv"),
        EnsureChannelFirstd(keys=image_keys),
        #Orientationd(keys=["image"], axcodes="RAS"),
        #Spacingd(keys=["image","agtv"], pixdim=(1,1,1),mode=("bilinear","nearest")),
        #ScaleIntensityRanged(keys=["image"], a_min=minmin_CT, a_max=maxmax_CT,b_min=0.0, b_max=1.0, clip=True,),
        Create_sequences(keys=image_keys),
        #CropForegroundd(keys=image_keys, source_key="lung",k_divisible = size),
        ToTensord(keys=image_keys),
    ]
)


keys to transpose: ('agtv',)


In [28]:
#Check the images after the preprocessing
if True:
    check_ds =Dataset(data=val_files, transform=val_transforms)
    check_loader = DataLoader(check_ds, batch_size=1,num_workers=0)
    if True:
        count = 1
        #for batch_data in check_loader:
        batch_data = first(check_loader)
        agtv = (batch_data["agtv"][0][0])
        #image,lung,agtv = (batch_data["image"][0][0],batch_data["lung"][0][0],batch_data["agtv"][0][0])
        #print(batch_data["image"].meta["filename_or_obj"][0])
        #print(f"px info:{count },image shape: {image.shape},lung shape: {lung.shape}")
        print(f"px info:{count },agtv shape: {agtv.shape}")
        count+=1
        if False:
            for i in range(image.shape[2]):
                plt.subplot(1,3,1),plt.imshow(image[:,:,i]),plt.axis('off')
                plt.subplot(1,3,2),plt.imshow(image[:,:,i]+lung[:,:,i]),plt.axis('off')
                plt.subplot(1,3,3),plt.imshow(image[:,:,i]+agtv[:,:,i]),plt.axis('off')
                count+=1
                plt.tight_layout(),plt.show()
                break
            

px info:1,agtv shape: (1, 288, 192)


In [ ]:
#Get the dataset ready for the model
if False:
    train_ds = Dataset(data=train_files, transform=train_transforms)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=0)

    val_ds = Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)#,collate_fn=pad_list_data_collate)

In [ ]:
#Create the model
spatial_dims = 3
max_epochs = 100
in_channels = 1
out_channels=2 #including background
if True:
    model = SwinUNETR(
        image_size, 
        in_channels, out_channels, 
        use_checkpoint=True, 
        feature_size=24,
        #spatial_dims=spatial_dims
    ).to(device)
else:
    model = UNet(
        spatial_dims=spatial_dims,
        in_channels=in_channels,
        out_channels=out_channels,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=4,
        norm=Norm.BATCH,
    ).to(device)

#metrics
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
dice_metric = DiceMetric(include_background=True, reduction="mean")
surfDice_metric = SurfaceDiceMetric([0.01,0.01],include_background=True)
surfDice_metric_1Class = SurfaceDiceMetric([3],include_background=False)
hausdorff_metric = HausdorffDistanceMetric(include_background=True, reduction="mean")

In [ ]:
# Load pretrained model
pretrained_path = '/home/umcg/Desktop/AutomaticITV_code/weights/best_m_MONAI_V3_NBIAWeightsretrainedWithUMCGdata.pth'

if pretrained_path is not(None):
    model.load_state_dict(torch.load(pretrained_path, map_location=torch.device(device)))

    #weight = torch.load(pretrained_path, map_location=torch.device(device))
    #model.load_from(weights=weight)
    print('Using pretrained weights!')

In [ ]:
post_transforms = Compose(
    [
        EnsureType(),
        AsDiscrete(argmax=True,threshold=.9),
        FillHoles(applied_labels=1, connectivity=3),
        RemoveSmallObjects(min_size=15, connectivity=3, independent_channels=False),
        KeepLargestConnectedComponent(applied_labels=1,num_components=3),
    ]
)

post_pred = Compose([EnsureType(), AsDiscrete(argmax=True,threshold=0.5)])
post_label = Compose([EnsureType(), AsDiscrete(threshold=0.5)],)


In [ ]:
#compute metric for current iteration
#Batch x Channel x Height x Width - [B,C,H,W] - 1, 384, 288, 192
#Channel is number of classes
def surfDiceFun_1Class(val_labels,postImg_monai):
    ylabe_BCHW_neg = val_labels[0].cpu().numpy()
    ylabe_BCHW_neg = ylabe_BCHW_neg*-1+1
    ylabe_BCHW_pos = val_labels[0].cpu().numpy()
    ylabe_BCHW = np.concatenate((ylabe_BCHW_neg, ylabe_BCHW_pos),0)
    transpose_monai = Compose([Transpose([3, 0, 1, 2])])
    ylabe_BCHW = transpose_monai(ylabe_BCHW)
    ypred_BCHW = postImg_monai[0].permute(3, 0, 1, 2)
    
    #print(ylabe_BCHW.shape,ypred_BCHW.shape)
    list_surf=[]
    for i in range(ypred_BCHW.shape[0]-4):
        if  np.sum(ylabe_BCHW[i:i+2,1,:,:])>0:# or np.sum(ypred_BCHW[i:i+2,0,:,:])>0:
            surfDice_metric_1Class(y_pred=ypred_BCHW[i:i+2,:,:,:], y=ylabe_BCHW[i:i+2,1:,:,:])
            list_surf.append(surfDice_metric_1Class.aggregate().item())
            #print(list_surf[-1])
            surfDice_metric_1Class.reset()
    return np.mean(list_surf)
#hausdorff_metric(y_pred=postImg_monai, y=val_labels)
#print('hausdorff monai    :',hausdorff_metric.aggregate().item())
#hausdorff_metric.reset()

In [ ]:
#Create headers to save the predictions
gtv = '/home/umcg/Desktop/AutomaticITV_code/MultipleBreathingP-OneDriveCopy/MultipleBreathingP/0082998/0082998_rtstruct_ITV.nii.gz'
gtv = nib.load(gtv)
empty_header = nib.Nifti1Header()


In [ ]:
print(bp)


In [ ]:
#Testing the model,no label used 
count=0
nr_images=6
figsize = (18, 12)
#figures_folder_i = '/data/p308104/NBIA_Data/NIFTI_NBIA/Res-16092022/' #Peregrine
figures_folder_i ='/home/umcg/Desktop/AutomaticITV_code/figures_folder_i/' #Local
all_metrics = []
model.eval()
with torch.no_grad():
    for val_data in check_loader:
        val_inputs = (val_data["image"].to(device))
        roi_size = image_size
        sw_batch_size = 1
        val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
        
        #postImg_manual  = PostProcessing(val_outputs.cpu())
        #postImg_manual  = [post_transforms(i) for i in decollate_batch(postImg_manual)]
        postImg_monai = [post_transforms(i) for i in decollate_batch(val_outputs)]
        val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
        px = val_data["image"].meta["filename_or_obj"][0].split('/')[-2]
        newGTV_name = val_data["image"].meta["filename_or_obj"][0][:-9]+'aGTV.nii.gz'
        bp = val_data["image"].meta["filename_or_obj"][0].split('%')[-2][-2:]
        print(newGTV_name)

        if True: #AXIAL
            slice_indices = []   
            for i in range(val_inputs.shape[4]):
                if np.sum(val_outputs[0][0,:,:,i])>0:
                    slice_indices.append(i)
            if len(slice_indices) <nr_images:
                slice_indices.append(random.sample(range(1, val_inputs.shape[4]),nr_images-len(slice_indices))[0])
            else:
                slice_indices = random.sample(slice_indices, k=nr_images)
            instance = random.randint(0, val_inputs.shape[0] - 1)
            j = 1
            for i, idx in enumerate(slice_indices):
                j=1+i
                fig = plt.figure('Instance = {}'.format(instance), figsize=figsize)
                plt.subplot(3, nr_images, j),plt.title('AXIAL CT ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, :, idx], cmap='gray', vmin=0, vmax=1),plt.axis('off')
                plt.subplot(3, nr_images, nr_images+j),plt.title('Prediction ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, :, idx]+val_outputs[0].detach().cpu()[instance, :, :, idx]),plt.axis('off')
                plt.subplot(3, nr_images, 2*nr_images+j),plt.title('postImg Monai ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, :, idx]+postImg_monai[0].detach().cpu()[instance, :, :, idx]),plt.axis('off')
            plt.tight_layout(),plt.show()
            if not os.path.exists(os.path.join(figures_folder_i,px)):
                os.makedirs(os.path.join(figures_folder_i,px)) 
            plt.savefig(os.path.join(figures_folder_i,px,bp+'AXIAL final_{}.png'.format(px)))
        if True: #SAGITAL
            slice_indices = []   
            for i in range(val_inputs.shape[3]):
                if np.sum(val_outputs[0][0,:,i,:])>0:
                    slice_indices.append(i)
            if len(slice_indices) <nr_images:
                slice_indices.append(random.sample(range(1, val_inputs.shape[3]),nr_images-len(slice_indices))[0])
            else:
                slice_indices = random.sample(slice_indices, k=nr_images)
            instance = random.randint(0, val_inputs.shape[0] - 1)
            j = 1
            for i, idx in enumerate(slice_indices):
                j=1+i
                fig = plt.figure('Instance = {}'.format(instance), figsize=figsize)
                plt.subplot(3, nr_images, j),plt.title('SAGITAL CT ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, idx,:], cmap='gray', vmin=0, vmax=1),plt.axis('off')
                plt.subplot(3, nr_images, nr_images+j),plt.title('Prediction ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, idx,:]+val_outputs[0].detach().cpu()[instance, :,idx,:]),plt.axis('off')
                plt.subplot(3, nr_images, 2*nr_images+j),plt.title('postImg Monai ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, idx,:]+postImg_monai[0].detach().cpu()[instance, :,idx,:]),plt.axis('off')
            plt.tight_layout(),plt.show()
            if not os.path.exists(os.path.join(figures_folder_i,px)):
                os.makedirs(os.path.join(figures_folder_i,px)) 
            plt.savefig(os.path.join(figures_folder_i,px,bp+'SAGITAL final_{}.png'.format(px)))
        if True: #CORONAL
            slice_indices = []   
            for i in range(val_inputs.shape[2]):
                if np.sum(val_outputs[0][0,i,:,:])>0:
                    slice_indices.append(i)
            if len(slice_indices) <nr_images:
                slice_indices.append(random.sample(range(1, val_inputs.shape[2]),nr_images-len(slice_indices))[0])
            else:
                slice_indices = random.sample(slice_indices, k=nr_images)
            instance = random.randint(0, val_inputs.shape[0] - 1)
            j = 1
            for i, idx in enumerate(slice_indices):
                j=1+i
                fig = plt.figure('Instance = {}'.format(instance), figsize=figsize)
                plt.subplot(3, nr_images, j),plt.title('CORONAL CT ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, idx,:,:], cmap='gray', vmin=0, vmax=1),plt.axis('off')
                plt.subplot(3, nr_images, nr_images+j),plt.title('Prediction ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, idx,:,:]+val_outputs[0].detach().cpu()[instance,idx,:,:]),plt.axis('off')
                plt.subplot(3, nr_images, 2*nr_images+j),plt.title('postImg Monai ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0,idx,:,:]+postImg_monai[0].detach().cpu()[instance,idx,:,:]),plt.axis('off')
            plt.tight_layout()
            plt.show()
            if not os.path.exists(os.path.join(figures_folder_i,px)):
                os.makedirs(os.path.join(figures_folder_i,px)) 
            plt.savefig(os.path.join(figures_folder_i,px,bp+'CORONAL final_{}.png'.format(px)))
        
        #No METRICS - compute metric for current iteration
        
        predicted_ni = nib.Nifti1Image(postImg_monai[0].cpu().numpy(), gtv.affine,empty_header)
        nib.save(predicted_ni,newGTV_name)
        count+=1
        if count>9:
            break
#NO METRICS  - Save csv of metrics

In [ ]:
#train with lung mask 
#to create the ITV - sum the bp gtv and maybe give weights to some phases
#use tumor mask to also create the ITV 



In [ ]:
#Load csv of metrics

if False:
    import pandas as pd
    df = pd.read_csv(r'/home/umcg/Desktop/AutomaticITV_code/figures_folder_i/res.csv')
    print(df.mean())
    print(df.std())

In [ ]:
      hausdorff_metric(y_pred=postImg_monai, y=val_labels)
        print('hausdorff monai    :',hausdorff_metric.aggregate().item())
        hausdorff_metric.reset()

In [ ]:
from skimage.measure import label, regionprops
from skimage.morphology import dilation,disk,erosion
def PostProcessing(predicted):
    img = predicted
    for i in range(predicted.shape[-1]-10):
        j=i+5
        #Get imgs
        Before = predicted[0,1,:,:,j-1]
        Current = predicted[0,1,:,:,j]
        After = predicted[0,1,:,:,j+1]
        AAfter = predicted[0,1,:,:,j+2]+After
        AAfter = predicted[0,1,:,:,j+3]+AAfter
        AAfter = predicted[0,1,:,:,j+4]+AAfter
        AAfter = predicted[0,1,:,:,j+5]+AAfter
        BBefore = predicted[0,1,:,:,j-2]+Before
        BBefore = predicted[0,1,:,:,j-3]+BBefore
        BBefore = predicted[0,1,:,:,j-4]+BBefore
        BBefore = predicted[0,1,:,:,j-5]+BBefore
        
        #Binarize
        t = .5
        BBefore[BBefore>=t] =int(1)
        BBefore[BBefore<t] =int(0)
        Current[Current>=t] =int(1)
        Current[Current<t] =int(0)
        AAfter[AAfter>=t] =int(1)
        AAfter[AAfter<t] =int(0)
        
        #Morphology
        m = 15
        BBefore = dilation(BBefore, disk(m))
        Current = dilation(Current, disk(m))
        AAfter = dilation(AAfter, disk(m))
        m = 5
        BBefore = erosion(BBefore, disk(m))
        Current = erosion(Current, disk(m))
        AAfter = erosion(AAfter, disk(m))
        
        #And Between slices -> <- ... for missing 
        ABS = np.logical_and(BBefore,AAfter)
        Current = np.logical_or(ABS,Current)
        
        #And outside slices <-->  ... for false positives with RegionProps
        BBefore_label = label(BBefore)
        Current_label = label(Current)
        AAfter_label = label(AAfter)
        BBefore_regions = regionprops(BBefore_label)
        Current_regions = regionprops(Current_label)
        AAfter_regions = regionprops(AAfter_label)
        #Look for centroids distance
        limit =  20
        if False:
            for c_centroid in Current_regions:
                for b_centroid in BBefore_regions:
                    dif = np.abs(np.asarray(c_centroid['centroid'])-np.asarray(b_centroid['centroid']))
                    if dif[0] > limit or dif[1] > limit: 
                        Current[Current==c_centroid['label']] = int(0)
                        print('delete blob')
                for a_centroid in AAfter_regions:
                    dif = np.abs(np.asarray(c_centroid['centroid'])-np.asarray(a_centroid['centroid']))
                    if dif[0] > limit or dif[1] > limit: 
                        Current[Current==c_centroid['label']] = int(0)
                        print('delete blob')
        if False:
            plt.subplot(1,4,1),plt.imshow(BBefore,cmap='gray', vmin=0, vmax=1)
            plt.subplot(1,4,2),plt.imshow(Current,cmap='gray', vmin=0, vmax=1)
            plt.subplot(1,4,3),plt.imshow(predicted[0,1,:,:,j],cmap='gray', vmin=0, vmax=1)
            plt.subplot(1,4,4),plt.imshow(AAfter,cmap='gray', vmin=0, vmax=1)
            plt.show()
    
        img[0,0,:,:,j] = torch.from_numpy(Current)
    return img



In [ ]:
                if "20%_ct.nii.gz" in f.lower():
                    CT20_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "20%_lungmask.nii.gz" in f.lower():
                    Lung20_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "30%_ct.nii.gz" in f.lower():
                    CT30_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "30%_lungmask.nii.gz" in f.lower():
                    Lung30_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "40%_ct.nii.gz" in f.lower():
                    CT40_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "40%_lungmask.nii.gz" in f.lower():
                    Lung40_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if  "50%_ct.nii.gz" in f.lower():
                    CT50_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "50%_lungmask.nii.gz" in f.lower():
                    Lung50_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "60%_ct.nii.gz" in f.lower():
                    CT60_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "60%_lungmask.nii.gz" in f.lower():
                    Lung60_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "70%_ct.nii.gz" in f.lower():
                    CT70_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "70%_lungmask.nii.gz" in f.lower():
                    Lung70_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "80%_ct.nii.gz" in f.lower():
                    CT80_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "80%_lungmask.nii.gz" in f.lower():
                    Lung80_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "90%_ct.nii.gz" in f.lower():
                    CT90_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "90%_lungmask.nii.gz" in f.lower():
                    Lung90_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "_ 0%_ct.nii.gz" in f.lower():
                    CT100_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "_ 0%_lungmask.nii.gz" in f.lower():
                    Lung100_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1
                if "itv.nii.gz" in f.lower():
                    lbl_fpaths.append(os.path.join(root_path,patient_path,f))
                    flag_PxOk+=1